In [1]:
import os
os.chdir('../')

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import numpy as np
from search.search import SearchEngine
from extract import *

In [3]:
%%time
se = SearchEngine()
entities = read_entities('documents.json')

se.bulk_index_entities(entities)

CPU times: user 589 ms, sys: 10.4 ms, total: 599 ms
Wall time: 601 ms


In [4]:
se.smart_search('банк')

[('https://cbr.ru/hd_base/infodirectrepousd/@-580919849936945116', 1),
 ('https://cbr.ru/banking_sector/credit/coinfo/?id=870000001@-6414058805706726457',
  1),
 ('https://cbr.ru/DKP/iubk/iubk_11-1-2/@1304801707148938670', 1),
 ('https://cbr.ru/analytics/bank_system/Exp12_12/@-8164793942298360177', 1),
 ('https://cbr.ru/analytics/bnksyst/ITM_41321/PUB_120101/@-8664685886725545137',
  1),
 ('https://cbr.ru/analytics/bank_system/Exp13_02/@-8832176026717747571', 1),
 ('https://cbr.ru/analytics/bank_system/din_razv_16_03/@5076132852118113198',
  1),
 ('https://cbr.ru/DKP/iubk/iubk_13-1-2/@-5454501505377454805', 1),
 ('https://cbr.ru/analytics/bank_system/din_razv_15_10/@4461968980026607861',
  1),
 ('https://cbr.ru/analytics/bank_system/Exp12_08/@1213491898646144898', 1)]

In [4]:
import pickle

In [6]:
se

In [8]:
pickle.dump(se, open('search.pkl', 'wb'))

In [5]:
loaded_se = pickle.load(open('search.pkl', 'rb'))
loaded_se

In [6]:
loaded_se.search('банк')

[('https://cbr.ru/banking_sector/credit/FullCoList/@11697539444930777929',
  0.15355627043642192),
 ('https://cbr.ru/registries/nps/oper_zip/@11990872770296143492',
  0.14696566609751535),
 ('https://cbr.ru/hd_base/infodirectrepousd/@17865824223772606500',
  0.1383504373952472),
 ('https://cbr.ru/banking_sector/credit/coinfo/?id=350000004@9958357458796590883',
  0.1383504373952472),
 ('https://cbr.ru/hd_base/infodirectreporub/@3087914180279970663',
  0.13226909948776383),
 ('https://cbr.ru/hd_base/bankpapers/@16027420484538685728',
  0.11222832683810263),
 ('https://cbr.ru/banking_sector/credit/coinfo/?id=460000022@9579957001126118664',
  0.10287596626826073),
 ('https://cbr.ru/banking_sector/credit/coinfo/?id=450000203@11681252705765210661',
  0.10287596626826073),
 ('https://cbr.ru/analytics/insideinfo/@14438311283264946717',
  0.10287596626826073),
 ('https://cbr.ru/analytics/insideinfo/@431226185384911881',
  0.10287596626826073)]

- по всем словам считать idf
- по всем документам посчитать среднюю длину
- tf по одному документу
- длина документа

In [74]:
def calc_factors(query, doc):
    static_factors = calc_static_factors(doc)
    dynamic_factors = calc_dynamic_factors('банк', doc)

    factors = static_factors | dynamic_factors
    return factors

def factors_to_list(query, url, factors):
    ordered_factors = sorted(factors.items(), key=lambda x: x[0])
    factor_values = list(map(lambda x: x[1], ordered_factors))
    return [query, url] + factor_values

In [6]:
query = 'банк'
words = normalize_string(query).split()
candidates = se.get_candidates(words)

candidates_to_rank = []
for candidate in candidates:
    doc = se._documents[candidate]
    factors = calc_factors(words, doc)
    factors = factors_to_list(query, candidate, factors)
    candidates_to_rank.append(factors)

In [8]:
candidates_to_rank[0]

[autoreload of search.rank failed: Traceback (most recent call last):
  File "/Users/svkov42/miniconda3/envs/ds/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/Users/svkov42/miniconda3/envs/ds/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 475, in superreload
    module = reload(module)
  File "/Users/svkov42/miniconda3/envs/ds/lib/python3.9/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 613, in _exec
  File "<frozen importlib._bootstrap_external>", line 846, in exec_module
  File "<frozen importlib._bootstrap_external>", line 983, in get_code
  File "<frozen importlib._bootstrap_external>", line 913, in source_to_code
  File "<frozen importlib._bootstrap>", line 228, in _call_with_frames_removed
  File "/Users/svkov42/Documents/Severstal_ICT2024/analytics/search/rank.py", line 6
    return sorted(candi

['банк',
 'https://cbr.ru/analytics/bank_system/Exp12_02/@-8810276713486921242',
 0,
 0,
 0,
 1.1,
 2.7333333333333334,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0.0,
 2.1666666666666665]

In [16]:
se.search('банк')

[('https://cbr.ru/banking_sector/credit/FullCoList/@-7638025034285683774',
  0.15355627043642192),
 ('https://cbr.ru/registries/nps/oper_zip/@8801460840335433866',
  0.14696566609751535),
 ('https://cbr.ru/hd_base/infodirectrepousd/@-580919849936945116',
  0.1383504373952472),
 ('https://cbr.ru/banking_sector/credit/coinfo/?id=350000004@-7024088650001845899',
  0.1383504373952472),
 ('https://cbr.ru/hd_base/infodirectreporub/@-7255443478235502537',
  0.13226909948776383),
 ('https://cbr.ru/hd_base/bankpapers/@-2831756634420824408',
  0.11222832683810263),
 ('https://cbr.ru/banking_sector/credit/coinfo/?id=460000022@-7402489107672318118',
  0.10287596626826073),
 ('https://cbr.ru/banking_sector/credit/coinfo/?id=450000203@-5301193403033226121',
  0.10287596626826073),
 ('https://cbr.ru/analytics/insideinfo/@-4008432790444604899',
  0.10287596626826073),
 ('https://cbr.ru/analytics/insideinfo/@431226185384911881',
  0.10287596626826073)]

# Факторы

## Статические

- is_cbr
- is_wiki
- is_html_table
- is_pdf_table
- is_series - временной ряд или нет
- float_ratio
- datetime_ratio
- str_ratio

## Динамические

- bm25
- idf
- bm25 - синонимы
- log_word_count - логарифм количества слов в запросе 
- is_report - если есть слово "отчет"

In [19]:
def calc_static_factors(entity):
    return {
        'is_cbr': calc_is_cbr(entity),
        'is_wiki': calc_is_wiki(entity),
        'is_html_table': calc_is_html_table(entity),
        'is_pdf_table': calc_is_pdf_table(entity),
        'is_series': calc_is_series(entity),
        'float_ratio': calc_float_ratio(entity),
        'datetime_ratio': calc_datetime_ratio(entity),
        'str_ratio': calc_str_ratio(entity),
    }

In [61]:
def calc_is_cbr(entity):
    return 1

def calc_is_wiki(entity):
    return 1

def calc_is_html_table(entity):
    return 1

def calc_is_pdf_table(entity):
    return 1

def calc_is_series(entity):
    return 1

def calc_float_ratio(entity):
    return entity.get('float_per_row', 0)

def calc_datetime_ratio(entity):
    return entity.get('datetime_per_row', 0)

def calc_str_ratio(entity):
    return entity.get('str_per_row', 0)

In [62]:
calc_static_factors(entities[0])

{'is_cbr': 1,
 'is_wiki': 1,
 'is_html_table': 1,
 'is_pdf_table': 1,
 'is_series': 1,
 'float_ratio': 2.8136094674556213,
 'datetime_ratio': 1.1198224852071006,
 'str_ratio': 3.587278106508876}

In [63]:
def calc_dynamic_factors(query, entity):
    words = normalize_string(query).split()
    return {
        'bm25': calc_bm25(words, entity),
        'idf_sum': calc_idf_sum(words, entity),
        'bm25sy': calc_bm25sy(words, entity),
        'bm25tr': calc_bm25tr(words, entity),
        'log_word_count': calc_log_word_count(words, entity),
        'is_report': calc_is_report(words, entity),
    }

In [64]:

def calc_bm25(words, entity, words_freq, idf, advl, k1=1.5, b=0.75):
    bm25 = 0
    tf = 0
    entity = normalize_entity(entity).split()
    for i, word in enumerate(words):
        
        freq = words_freq[i]
        numerator = freq * (k1 + 1)
        denominator = freq + k1 * (
            1 - b + b * len(entity) / avdl
        )
        idf_score * numerator / denominator
    return 0

def calc_idf_sum(words, entity):
    return 0

def calc_bm25sy(words, entity):
    return 0

def calc_bm25tr(words, entity):
    """bm25 translated"""
    return 0

def calc_log_word_count(words, entity):
    return np.log(len(words))

def calc_is_report(words, entity):
    return 0

In [65]:
calc_dynamic_factors('банк яндекс', entities)

{'bm25': 0,
 'idf_sum': 0,
 'bm25sy': 0,
 'bm25tr': 0,
 'log_word_count': 0.6931471805599453,
 'is_report': 0}